In [15]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#import API key
from config import g_key

#access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
#import cities.csv file into a dataframe
cities_df = pd.read_csv("cities.csv")

#get rid of 'City_ID' column
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Mumford,73,GH,1579838828,94,5.26,-0.76,76.96,3.36
3,Omboue,16,GA,1579838830,95,-1.57,9.26,75.83,0.36
4,Jamestown,90,US,1579838791,43,42.10,-79.24,39.00,5.82
5,Takoradze,2,GH,1579838832,93,4.88,-1.76,77.79,2.55
6,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51
7,Winneba,20,GH,1579838834,88,5.35,-0.62,82.40,4.70
8,Teshie,20,GH,1579838835,88,5.58,-0.11,82.40,4.70


In [3]:
#create heatmap based on humidity

#store latitude and longitude into the variable 'coordinates'
coordinates = cities_df[["Lat", "Lng"]]

#convert humidity to float; all data read from a csv is read as a string
humidity = cities_df["Humidity"].astype(float)

In [4]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Mumford,73,GH,1579838828,94,5.26,-0.76,76.96,3.36
3,Omboue,16,GA,1579838830,95,-1.57,9.26,75.83,0.36
4,Jamestown,90,US,1579838791,43,42.10,-79.24,39.00,5.82
5,Takoradze,2,GH,1579838832,93,4.88,-1.76,77.79,2.55
6,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51
7,Winneba,20,GH,1579838834,88,5.35,-0.62,82.40,4.70
8,Teshie,20,GH,1579838835,88,5.58,-0.11,82.40,4.70


In [12]:
specs_df = cities_df.loc[(cities_df["Max Temp"] > 40) &
                        (cities_df["Humidity"] < 93) & 
                        (cities_df["Wind Speed"] < 3) | 
                        (cities_df["Wind Speed"] > 8), :].reset_index()
specs_df = specs_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
specs_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51


In [13]:
nan_df = specs_df[specs_df.isna().any(axis=1)]
nan_df

hotel_df = specs_df.dropna()
hotel_df

#try this on the larger dataframe
#cities_nan = cities_df[cities_df.isna().any(axis=1)]
#cities_nan
#if there are nan values, then save by coding cities_df = cities_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51


In [14]:
#set up additional columns to hold information
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57,
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24,
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51,


In [42]:
# params dictionary to update each iteration
target_search = "Hotel"

params = {
    "keyword": target_search,
    "radius": 5000,
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_request = requests.get(base_url, params=params)
    
#    print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(hotel_request.url)

    # convert to json
    hotel_request = hotel_request.json()
    #print(json.dumps(hotel_request, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_request["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df
        

Missing field/result... skipping.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57,Jimmy Hotel
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24,BeatzWave Studios
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51,
